# L1: Information Retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open("app-descriptions.json.bz2") as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. Data in a DataFrame can be accessed in various ways, including by row and by column. To give an example, the code in the next cell shows rows 200–204:

In [2]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to acess fields from the description column.

In [3]:
df["description"][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any kind of transformation that is applied before a text is vectorized. Here you can restrict yourself to a very simple preprocessing: tokenization, stop word removal, and lemmatization.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure that you read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information that you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you feel necessary.

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=["tagger","parser","ner"])

def preprocess(text):
    pre = []
    for token in nlp(text):
        if token.is_stop == False and token.is_alpha == True:
            pre.append(token.lemma_)
    return pre

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses given text by tokenizing it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

**Tip:** To speed up the preprocessing, you can disable loading those spaCy components that you do not need, such as the part-of-speech tagger, parser, and named entity recognizer. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

Test your implementation by running the following cell:

In [5]:
preprocess("Apple is looking at buying U.K. startup for $1 billion")

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorizing

Your next task is to vectorize the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectorizer.

In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=preprocess)

X = vectorizer.fit_transform(df["description"])

Test your implementation by running the following cell:

In [7]:
X.shape

(1614, 20663)

This should show the dimensions of the matrix `X` to be 1614 × 20651.

## Problem 3: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms have the lowest/highest idf with respect to the app descriptions.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [16]:
idfs_sorted = vectorizer.idf_.argsort()
terms = feature_names[idfs_sorted].values.tolist()[0]
print("Most common words", terms[:10],
      "\nLeast common words", terms[-10:])

Most common words ['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'well'] 
Least common words ['chibi', 'chiamato', 'outward', 'chia', 'outworld', 'chew', 'overblown', 'overboard', 'outplay', 'ﬂying']


In [17]:
# TODO: Replace the next line with your own code.
feature_names = pd.DataFrame(vectorizer.get_feature_names()).transpose()
X_comp = X.toarray().sum(axis=0)
X_comp_sorted = X_comp.argsort()
print(X_comp_sorted.max(), X_comp_sorted.min(), feature_names.shape)
terms = feature_names[X_comp_sorted].values.tolist()[0]
len(terms)

20662 0 (1, 20663)


20663

Now, print out the 10 terms at the top/end of the sorted list:

In [18]:
print("Least common words", terms[:10],
      "\nMost common words", terms[-10:])

Least common words ['snabböverföring', 'signerar', 'nummer', 'bolåneräntor', 'bolån', 'fasträntekonto', 'värdebesked', 'filtrera', 'autogiro', 'samverkar'] 
Most common words ['level', 'fun', 'world', 'free', 'new', 'puzzle', 'kid', 'app', 'play', 'game']


## Problem 4: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to do solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning, and allows you to easily find a predefined number of app descriptions whose vector representations are closest in distance to the query vector.

In [19]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(10, metric="cosine")
model.fit(X.todense())

def search(query):
    nearestNeighbours = model.kneighbors(vectorizer.transform([query]).todense(), return_distance=False).reshape((10))
    return df['name'][nearestNeighbours]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

Test your implementation by running the following cell:

In [20]:
search("dodge trains")

1301                               Subway Surfers
1300                       Subway Princess Runner
1428                        Train Conductor World
998                No Humanity - The Hardest Game
1394                                   Tiny Rails
1429          Train for Animals - BabyMagica free
1168                                         Rush
1077                  Polar Flow – Sync & Analyze
1286    Strava: Track Running, Cycling & Swimming
1465              Virus War - Space Shooting Game
Name: name, dtype: object

The top hit in the list should be *Subway Surfers*.

## Problem 5: Keyword extraction

A simple method for extracting salient keywords from a document is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a given text.

In [21]:
def keywords(text, n=10):
    vec = (vectorizer.transform([text]).toarray()*-1).argsort().reshape(-1)
    return feature_names[vec[:n]].values.tolist()[0]

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

Test your implementation by running the following cell:

In [22]:
print(keywords(df["description"][1428]))

['train', 'railroad', 'railway', 'rail', 'chaos', 'crash', 'haul', 'railyard', 'timetable', 'tram']


This should give the following output:

<div class="alert alert-info">
    Please read the General information section on the lab web page before submitting this notebook!
</div>